# Taxonomic analysis 

Preambule to Multiple Sequence alignment and Phylogenetic tree building.

### Import necessary modules 

In [24]:
from Bio import (
    SeqIO as seqio, 
    SearchIO as searchio, 
    Entrez as entrez
)

from Bio.Seq import Seq as seq

import toml

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from functools import partial 

from annotathon.utils.customobjs import (
    Path as path,
    objdict as odict
)

from annotathon.parsing.blast import parse_alignment_descriptions as parse_ncbi

### Configuration to access NCBI's servers :

In [48]:
# Load configuration to access NCBI :
with open("../creds/entrezpy.toml", "r") as f:
    ncbi = toml.load(f, _dict=odict)

In [41]:
# set credentials : 
entrez.api_key = ncbi.credentials.apikey
entrez.email = ncbi.credentials.email
entrez.tool = ncbi.credentials.tool

In [47]:
# set plotting params :
%matplotlib inline
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (15, 8)

In [52]:
with open("../config/locations.toml", "r") as f:
    _config = toml.load(f, _dict=odict)
_config

{'locations': {'default': '../data/homologues-par-blast/default-nr',
  'cinqk': '../data/homologues-par-blast/5k-nr',
  'landmark': '../data/homologues-par-blast/landmark-noexclude',
  'sp': '../data/homologues-par-blast/swissprot',
  'anthony': '../data/anthony/'}}

In [56]:
locations = odict({ 
    key: path(value) for key, value in _config.locations.items()
})
locations

{'default': PosixPath('../data/homologues-par-blast/default-nr'),
 'cinqk': PosixPath('../data/homologues-par-blast/5k-nr'),
 'landmark': PosixPath('../data/homologues-par-blast/landmark-noexclude'),
 'sp': PosixPath('../data/homologues-par-blast/swissprot'),
 'anthony': PosixPath('../data/anthony')}

In [58]:
blast = odict({
    "locations": locations,
    "data": odict({})
})
blast

{'locations': {'default': PosixPath('../data/homologues-par-blast/default-nr'),
  'cinqk': PosixPath('../data/homologues-par-blast/5k-nr'),
  'landmark': PosixPath('../data/homologues-par-blast/landmark-noexclude'),
  'sp': PosixPath('../data/homologues-par-blast/swissprot'),
  'anthony': PosixPath('../data/anthony')},
 'data': {}}

In [59]:
description_glob = "*Alignment-Descriptions*"

In [60]:
blast.data.update(odict({
    "default": parse_ncbi(blast.locations.default.lglob(description_glob)[0]),
    "cinqk": parse_ncbi(blast.locations.cinqk.lglob(description_glob)[0]),
    "landmark": parse_ncbi(blast.locations.landmark.lglob(description_glob)[0]),
    "sp": parse_ncbi(blast.locations.sp.lglob(description_glob)[0]),
    "taxo": parse_ncbi(blast.locations.anthony.lglob(description_glob)[0]),
    "hypo": parse_ncbi(blast.locations.anthony.lglob("*.csv")[1])
}))

In [32]:
tests = pd.read_csv("efetch-tests.csv")

In [27]:
with entrez.esearch(db="pubmed", term="biopython") as wow:
    x = entrez.read(wow)

In [44]:
with entrez.efetch(db="protein", id=tests.Accession.to_list(), rettype="gb", retmode="text") as finallyyeah:
    y = [ entry for entry in seqio.parse(finallyyeah, format="gb") ]

In [45]:
for i in y:
    print(i.annotations["taxonomy"])

['Bacteria', 'Proteobacteria', 'Alphaproteobacteria', 'Pelagibacterales', 'unclassified Pelagibacterales']
[]
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Firmicutes', 'Erysipelotrichia', 'Erysipelotrichales', 'Erysipelotrichaceae', 'Turicibacter']
['Bacteria', 'Firmicutes', 'Erysipelotrichia', 'Erysipelotrichales', 'Erysipelotrichaceae', 'Turicibacter']
['Bacteria', 'Firmicutes', 'Erysipelotrichia', 'Erysipelotrichales', 'Erysipelotrichaceae', 'Turicibacter']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']


In [ ]:
with open("cor6_6.gb", "rU") as input_handle, open(
    "cor6_6.fasta", "w"
) as output_handle:
    sequences = SeqIO.parse(input_handle, "genbank")
    count = SeqIO.write(sequences, output_handle, "fasta")

In [69]:
# SAVE AS GENEBANK ! 
# FASTA LOSES LOTS OF INFORMATION !

In [68]:
with entrez.efetch(db="protein", id=tests.Accession.to_list(), rettype="gb", retmode="text") as in_handle:
    with open("gb-out-test.gb", "w") as out_handle:
        with open("fasta-out-test.fasta", "w") as o2:
            sequences = seqio.parse(in_handle, format="gb")
            #seqio.write(sequences, out_handle, format="gb") 
            seqio.write(sequences, o2, format="fasta") 

In [61]:
# some dummy test : 
with entrez.efetch(db="nucleotide", id="EU490707", rettype="gb", retmode="text") as wow:
    print(wow.read())

LOCUS       EU490707                1302 bp    DNA     linear   PLN 26-JUL-2016
DEFINITION  Selenipedium aequinoctiale maturase K (matK) gene, partial cds;
            chloroplast.
ACCESSION   EU490707
VERSION     EU490707.1
KEYWORDS    .
SOURCE      chloroplast Selenipedium aequinoctiale
  ORGANISM  Selenipedium aequinoctiale
            Eukaryota; Viridiplantae; Streptophyta; Embryophyta; Tracheophyta;
            Spermatophyta; Magnoliopsida; Liliopsida; Asparagales; Orchidaceae;
            Cypripedioideae; Selenipedium.
REFERENCE   1  (bases 1 to 1302)
  AUTHORS   Neubig,K.M., Whitten,W.M., Carlsward,B.S., Blanco,M.A., Endara,L.,
            Williams,N.H. and Moore,M.
  TITLE     Phylogenetic utility of ycf1 in orchids: a plastid gene more
            variable than matK
  JOURNAL   Plant Syst. Evol. 277 (1-2), 75-84 (2009)
REFERENCE   2  (bases 1 to 1302)
  AUTHORS   Neubig,K.M., Whitten,W.M., Carlsward,B.S., Blanco,M.A.,
            Endara,C.L., Williams,N.H. and Moore,M.J.
  TIT

In [18]:
help(entrez.efetch)

Help on function efetch in module Bio.Entrez:

efetch(db, **keywords)
    Fetch Entrez results which are returned as a handle.
    
    EFetch retrieves records in the requested format from a list or set of one or
    more UIs or from user's environment.
    
    See the online documentation for an explanation of the parameters:
    http://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch
    
    Return a handle to the results.
    
    Raises an IOError exception if there's a network error.
    
    Short example:
    
    >>> from Bio import Entrez
    >>> Entrez.email = "Your.Name.Here@example.org"
    >>> handle = Entrez.efetch(db="nucleotide", id="AY851612", rettype="gb", retmode="text")
    >>> print(handle.readline().strip())
    LOCUS       AY851612                 892 bp    DNA     linear   PLN 10-APR-2007
    >>> handle.close()
    
    This will automatically use an HTTP POST rather than HTTP GET if there
    are over 200 identifiers as recommended by the NCBI.
    
    